## Functions

In [44]:
import LyndonWords
import importlib
importlib.reload(LyndonWords)
from LyndonWords import *
import itertools
from functools import partial
import multiprocessing as mp
from tqdm.notebook import tqdm
import math
import numpy as np
def genPermutations(type,n,k):
    with mp.Pool() as pool:
        for i in tqdm(pool.imap_unordered(partial(rootSystem,type=type,k=k),itertools.permutations(np.array(range(n+1)))),total=math.factorial(n+1)):
            yield i

In [45]:
def monotonicityConj(rootsys:rootSystem,wordToCheck:word):
    wordPlusDelta = rootsys.getWords(wordToCheck.weights + wordToCheck.delta)[0]
    parsedFormat = rootsys.parseToDeltaFormat(wordPlusDelta)
    if(type(parsedFormat[0]) is list):
        return True
    return False
#TODO: Updated bases on new claim on monotonicity
def checkMonotoneIncreasingImpCond(rootSystemPermutations):
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.getWords(rootsys.delta)
        temp = rootsys.checkMonotonicity("Increasing")
        monotoneIncreasing = [rootsys.getWords(i[0] + rootsys.delta)[0] for i in temp]
        deltaArrays = [i.string for i in deltaWords]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(not str(i).startswith(str(j)) and (i.weights - rootsys.delta)[rootsys.ordering[0].rootIndex-1] > 0):
                    flag = True
                    break
            if(flag):
                continue
            returnArr.append((str(rootsys.ordering),(i.weights - rootsys.delta)))
    return returnArr

In [46]:
def checkCondImpMonotoneIncreasing(rootSystemPermutations):
    returnArr = []
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.getWords(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.getWords(base + rootsys.delta)[0]
            if(base[rootsys.ordering[0].rootIndex-1] == 0):
                continue
            deltaStart = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaHeight] == i)):
                    deltaStart = True
                    break
            if deltaStart:
                continue
            if(rootsys.getMonotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.getWords(base)[0])))
    return returnArr

In [47]:
def printByLength(rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.getWords(comb):
                print(" "+str(i))

In [48]:
def printByBaseWord(rootSystem:rootSystem,comb):
    k=0
    isImaginary = rootSystem.isImaginary(comb)
    for i in rootSystem.getAffineWords(comb):
        if(isImaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not isImaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [49]:
def exportByLength(rootSystem,quoteMarks=False):
    file = open("out.txt",'w')
    file.write(str(rootSystem.ordering)+"\n")
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            for i in rootSystem.getWords(comb):
                if(quoteMarks):
                    file.write(f"'{str(i)}',")
                else:
                    file.write(str(i) + '\n')
    file.close()

In [50]:
def deltaSplitAtLast(rootsys:rootSystem,baseWord:word,deltaWord:word) -> bool:
    deltaHeight = baseWord.height
    splitting = rootsys.parseToDeltaFormat(deltaWord)
    return len(splitting[-1]) == 1

In [51]:
def deltaSplitAtCofac(rootsys: rootSystem,baseWord: word, deltaWord: word) -> bool:
    cofacSplit = rootsys.costfac(baseWord)[0].height
    return (word.letterListCmp(deltaWord[:cofacSplit], baseWord[:cofacSplit]) == 0
                    and word.letterListCmp(deltaWord[cofacSplit-rootsys.deltaHeight:], baseWord[cofacSplit:]) == 0)

In [52]:
def exceptionalDeltaPattern(rootsys: rootSystem, index: int) ->bool:
    deltaWords = rootsys.getAffineWords(rootsys.delta)[index::rootsys.n]
    splitting = [i.noCommas() for i in rootsys.costfac(deltaWords[1])]
    for i in deltaWords[2:]:
        parsed = rootsys.parseToDeltaFormat(i)
        if(len(parsed) != 3):
            return False
        if(parsed[0] != splitting[0] or parsed[2] != splitting[1] or type(parsed[1]) is not list):
            return False
    return True

In [53]:
def checkDeltaPattern(perms,k=2,printDelta = False):
    for i in perms:
        baseImaginaryWords = i.getWords(i.delta)
        deltaWords = i.getWords(k*i.delta)
        flag = True
        for j in range(len(deltaWords)):
            #Tests if split is at last word
            if(not deltaSplitAtLast(i,baseImaginaryWords[j],deltaWords[j])):
                cofacSplit = i.costfac(baseImaginaryWords[j])[0].height
                deltaWord = deltaWords[j]
                if(
                    deltaSplitAtCofac(i,baseImaginaryWords[j],deltaWords[j])
                   ):
                    matchingFlag = True
                    baseDeltaIndex = 0
                    for baseDelta in baseImaginaryWords:
                        deltaFlag = False
                        for deltaOff in range((len(deltaWord)//i.deltaHeight) - 1):
                            if(word.letterListCmp(deltaWord[cofacSplit+(deltaOff*i.deltaHeight):cofacSplit+(deltaOff+1)*i.deltaHeight],
                                                baseDelta.string) == 0):
                                deltaFlag = True
                                break
                        matchingFlag=deltaFlag
                        if(matchingFlag):
                            break
                        baseDeltaIndex+=1
                    if(matchingFlag):
                        continue
                if(flag):
                    print(i.ordering)
                    flag = False
                if(printDelta):
                    print(f"  SL_{j}(\delta): " + '-'.join([str(i) for i in i.costfac(i.getWords(i.delta)[j])]))
                print(f"  SL_{j}({k}*\delta): " + '-'.join([str(i) for i in i.costfac((deltaWords[j]))]))

In [54]:
def checkHDeltasRepeat(perms):
    excepts = []
    for i in perms:
        hs = [j.hs for j in i.getAffineWords(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

In [55]:
def compareRootSystems(r1,r2):
    words = list(r1.weightToWordDictionary.values())
    for i in words:
        for j in range(len(i)):
            if(str(i[j]) != str(r2.getWords(i[j].weights)[j])):
                yield (i[j].weights,str(i[j]),str(r2.getWords(i[j].weights)[j]))

In [56]:
def generateEquivalence(permutations):
    equivalences = np.arange(len(permutations))
    for i in range(len(permutations)):
        for j in range(0,i):
            if(equivalences[j] != j):
                continue
            flag = False
            for k in compareRootSystems(permutations[i],permutations[j]):
                flag =True
                break
            if(not flag):
                yield (permutations[i].ordering,permutations[j].ordering)

In [57]:
def getMaxPeriodicity(rootSystems):
    max = 0
    maxOrdering = ""
    maxRoot = []
    for rootsys in rootSystems:
        for simpleRoot in rootsys.baseWeights[:-1]:
            result = rootsys.getPeriodicity(simpleRoot)
            if(result > max):
                max = result
                maxOrdering = str(rootsys.ordering)
                maxRoot=simpleRoot
    return (max,maxOrdering,maxRoot)

In [58]:
class deltaTypes:
    def __init__(self,index:int,hs:list,type:str,insertedIndex:int,leftfac:str,rightfac:str):
        self.index = index
        self.hs = hs
        #is either "last" or "cofac" or "neither" if something goes horribly wrong
        self.type = type
        self.insertedIndex = insertedIndex
        self.leftfac = leftfac
        self.rightfac = rightfac
    def toList(self):
        return [self.index,self.type,self.insertedIndex,self.leftfac,self.rightfac]
class deltaTypesCollection:
    def __init__(self,rootsys:rootSystem,deltaTypesList):
        self.type = rootsys.type
        self.ordering = str(rootsys.ordering)
        self.deltaTypes = deltaTypesList
    def toCsv(self)->str:
        retstr = f"{self.type},{self.ordering},,,,\n"
        for i in self.deltaTypes:
            retstr+= f"{i.index},[{' '.join([str(j) for j in i.hs])}],{i.type},{i.insertedIndex},{i.leftfac},{i.rightfac}\n"
        return retstr
    def containsExceptional(self) ->bool:
        for i in self.deltaTypes:
            if i.type != "last":
                return True
        return False
    def containsExceptionalLessFirst(self) -> bool:
        for i in self.deltaTypes[1:]:
            if i.type != "last":
                return True
        return False

In [59]:
def GenerateDeltaTypes(rootsys:rootSystem,k=2) ->deltaTypesCollection:
    deltaWords = rootsys.getWords(rootsys.delta)
    listOfDeltaTypes = []
    kdeltaWords = rootsys.getWords(rootsys.delta*k)
    for i in range(rootsys.n):
        kdeltaWord = kdeltaWords[i]
        splitting = 0
        parsedForm = rootsys.parseToDeltaFormat(kdeltaWord)
        for j in parsedForm:
            if(type(j) is not str):
                splitting = j[0]
                break
        if(deltaSplitAtLast(rootsys,deltaWords[i],kdeltaWord)):
            breakType = "last"
        elif(deltaSplitAtCofac(rootsys,deltaWords[i],kdeltaWord)):
            breakType = "cofac"
        elif(exceptionalDeltaPattern(rootsys,i)):
            breakType = "exceptional"
        else:
            breakType = "neither"
        factors = rootsys.costfac(deltaWords[i])
        listOfDeltaTypes.append(deltaTypes(i+1,deltaWords[i].hs,breakType,splitting,factors[0].noCommas(),factors[1].noCommas()))
    return deltaTypesCollection(rootsys,listOfDeltaTypes)
        

In [60]:
def costfacDeltaTypeConditionsMet(rootsys,deltaWords,index):
    factorization = rootsys.costfac(deltaWords[index])
    for i in range(index):
        if(factorization[1] < deltaWords[i]):
            if(rootsys.eBracket(rootsys.costfac(deltaWords[index])[1] + deltaWords[i])):
                return True
        else:
            break
    minusarr = np.zeros(rootsys.n+1,dtype=int)
    minusarr[deltaWords[index][-1].rootIndex-1] += 1
    if(word.letterListCmp(rootsys.getWords(rootsys.delta - minusarr)[0].string,deltaWords[index][:-1]) != 0):
        return True
    for j in range(index):
        if(word.letterListCmp(rootsys.costfac(deltaWords[j])[0].string,
                        rootsys.costfac(deltaWords[index])[1].string[:-1]) == 0):
            return True
    return False
def lastSplitDeltaTypeConditionsMet(rootsys:rootSystem,deltaWords:list,index:int) -> bool:
    if(rootsys.delta[rootsys.ordering[0].rootIndex-1] == 1):
        return True
    rightfac = rootsys.costfac(deltaWords[index])[1]
    for deltaWord in deltaWords[:index]:
        if(word.letterListCmp(rootsys.costfac(deltaWord)[1].string,rightfac.string[:-1]) == 0):
            return True
    return False
def exceptionalDeltaTypeCondictionsMet(rootsys:rootSystem,deltaWords:list,index:int) -> bool:
    if(rootsys.type == 'A'):
        return False
    if(rootsys.type == 'B'):
        return False
    if(rootsys.type == 'C'):
        return False
    if(rootsys.type == 'D'):
        return False
    if(rootsys.type == 'E'):
        if(rootsys.n == 6):
            if(np.all(rootsys.costfac(deltaWords[index])[1].weights == np.array([0,1,2,1,0,1,1],dtype=int))):
                return True
            if(np.all(rootsys.costfac(deltaWords[index])[1].weights == np.array([1,1,2,1,0,1,0],dtype=int))):
                return True
            if(np.all(rootsys.costfac(deltaWords[index])[1].weights == np.array([0,1,2,1,1,1,0],dtype=int))):
                return True
            return False
    if(rootsys.type == 'F'):
        if(np.all(rootsys.costfac(deltaWords[index])[1].weights == np.array([1,2,2,0,1],dtype=int))):
            return True
        return False
    if(rootsys.type == 'G'):
        return False
def checkDeltaTypePrediction(rootsys:rootSystem,k=2) -> list:
    result = GenerateDeltaTypes(rootsys,k)
    exceptions = []
    deltaWords = rootsys.getWords(rootsys.delta)
    for i in range(len(result.deltaTypes)):
        if((result.deltaTypes[i].type == "cofac") != costfacDeltaTypeConditionsMet(rootsys,deltaWords,i)):
            exceptions.append((rootsys.type, str(rootsys.ordering),i+1,result.deltaTypes[i].type,"cofac"))
        #if((result.deltaTypes[i].type == "last") != lastSplitDeltaTypeConditionsMet(rootsys,deltaWords,i)):
        #    exceptions.append((rootsys.type, str(rootsys.ordering),i+1,result.deltaTypes[i].type,"last"))
        if((result.deltaTypes[i].type == "exceptional") != exceptionalDeltaTypeCondictionsMet(rootsys,deltaWords,i)):
           exceptions.append((rootsys.type, str(rootsys.ordering),i+1,result.deltaTypes[i].type,"exceptional"))    
    return exceptions
def checkDeltaTypePrectionPerms(rootsystems,k=2):
    exceptions = []
    for rootsys in rootsystems:
        exceptions.extend(checkDeltaTypePrediction(rootsys,k))
    return exceptions

In [61]:
def writeDeltaTypesToFile(rootSystems,file="out.txt",k=2,onlyIfExceptional=True):
    fileobj = open(file,'w')
    fileobj.write("imaginary index, bracketing, splitting type, inserted imaginary index, left factor, right factor\n")
    for i in rootSystems:
        deltaTypesCol = GenerateDeltaTypes(i)
        if(onlyIfExceptional):
            if(not deltaTypesCol.containsExceptionalLessFirst()):
                continue
        fileobj.write(deltaTypesCol.toCsv())
    fileobj.close()

## Type A

In [101]:
Aperms = genPermutations('A',6,2)

In [102]:
checkDeltaTypePrectionPerms(Aperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

[]

In [22]:
writeDeltaTypesToFile(Aperms,"ADeltaTypes.csv",onlyIfExceptional=False)

In [32]:
getMaxPeriodicity(Aperms)

(3, '0<2<1<3<4<5', array([1, 1, 1, 0, 0, 0]))

## Type C

In [106]:
Cperms = genPermutations('C',5,2)

In [105]:
checkDeltaTypePrectionPerms(Cperms)

  0%|          | 0/720 [00:00<?, ?it/s]

[('C', '1<4<0<2<3<5', 5, 'last', 'last'),
 ('C', '1<4<0<3<5<2', 5, 'last', 'last'),
 ('C', '1<4<0<2<5<3', 5, 'last', 'last'),
 ('C', '1<4<0<3<2<5', 5, 'last', 'last'),
 ('C', '1<4<0<5<3<2', 5, 'last', 'last'),
 ('C', '1<4<0<5<2<3', 5, 'last', 'last'),
 ('C', '1<4<5<0<2<3', 5, 'last', 'last'),
 ('C', '1<4<5<0<3<2', 5, 'last', 'last'),
 ('C', '1<5<0<2<4<3', 4, 'last', 'last'),
 ('C', '1<5<0<2<4<3', 5, 'last', 'last'),
 ('C', '1<5<0<2<3<4', 4, 'last', 'last'),
 ('C', '1<5<0<2<3<4', 5, 'last', 'last'),
 ('C', '1<5<0<3<4<2', 4, 'last', 'last'),
 ('C', '1<5<0<3<4<2', 5, 'last', 'last'),
 ('C', '1<5<0<4<2<3', 4, 'last', 'last'),
 ('C', '1<5<0<4<2<3', 5, 'last', 'last'),
 ('C', '1<5<0<3<2<4', 4, 'last', 'last'),
 ('C', '1<5<0<3<2<4', 5, 'last', 'last'),
 ('C', '1<5<0<4<3<2', 4, 'last', 'last'),
 ('C', '1<5<0<4<3<2', 5, 'last', 'last'),
 ('C', '1<5<4<0<2<3', 5, 'last', 'last'),
 ('C', '1<5<4<0<3<2', 5, 'last', 'last'),
 ('C', '2<5<0<3<1<4', 4, 'last', 'last'),
 ('C', '2<5<0<3<4<1', 4, 'last', '

In [107]:
writeDeltaTypesToFile(Cperms,"CDeltaTypes.csv",onlyIfExceptional=False)

  0%|          | 0/720 [00:00<?, ?it/s]

In [21]:
print(*[f"{str(i[0])}: {i[1]}\n" for i in checkMonotoneIncreasingImpCond(Cperms)])

5<4<3<2<1<0: [0 0 0 0 0 1]
 5<4<3<2<1<0: [1 0 0 0 0 1]
 5<4<3<2<1<0: [1 1 0 0 0 1]
 5<4<3<2<1<0: [2 0 0 0 0 1]
 5<4<3<2<1<0: [2 1 0 0 0 1]
 5<4<3<2<1<0: [2 2 0 0 0 1]



In [22]:
checkCondImpMonotoneIncreasing(Cperms)

[('1<3<0<2<4<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<2<5<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<4<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<4<5<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<5<2<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<5<4<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<0<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<0<5<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<5<0<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<0<2<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<0<4<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<4<0<2', '1,2,0,3,4,5,4,3,2'),
 ('1<4<0<2<3<5', '1,2,3,0,4,5,4,3'),
 ('1<4<0<2<3<5', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<2<5<3', '1,2,3,0,4,5,4,3'),
 ('1<4<0<2<5<3', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<3<2<5', '1,2,3,0,4,5,4,3'),
 ('1<4<0<3<2<5', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<3<5<2', '1,2,3,0,4,5,4,3'),
 ('1<4<0<3<5<2', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<5<2<3', '1,2,3,0,4,5,4,3'),
 ('1<4<0<5<2<3', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<5<3<2', '1,2,3,0,4,5,4,3'),
 ('1<4<0<5<3<2', '1,2,3,0,4,5,4,3,2'),
 ('1<4<3<0<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<4<3<0<5<2', '1,2,0,3,4,5,4,3,2')

In [20]:
checkDeltaPattern(Cperms,2,printDelta=True)

In [25]:
checkHDeltasRepeat(Cperms)

[]

In [26]:
index=250
print(Cperms[index].ordering)
[print(*list(Cperms[index].costfac(i))+[i.hs],sep='-',) for i in Cperms[index].getAffineWords(Cperms[index].delta)]

2<0<3<5<1<4
2,1,3,4,0-2,1,3,4,5-[1 1 1 1 1]
2,1,3,4-2,1,3,4,5,0-[1 1 1 1 0]
2,1,3,0-2,1,3,4,5,4-[1 1 1 2 1]
2,1,0-2,1,3,4,5,4,3-[1 1 2 2 1]
2,3,4,5,4,3-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [27]:
for i in generateEquivalence(Cperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<0<4<5 

In [86]:
writeDeltaTypesToFile(Cperms)

## Type G

In [55]:
G2perms = genPermutations('G',2,5)

In [52]:
checkDeltaTypePrectionPerms(G2perms)

  0%|          | 0/6 [00:00<?, ?it/s]

[('G', '0<2<1', 2, 'last', 'last'),
 ('G', '2<0<1', 2, 'last', 'last'),
 ('G', '2<1<0', 2, 'last', 'last')]

In [56]:
writeDeltaTypesToFile(G2perms,"GDeltaTypes.csv",onlyIfExceptional=False)

  0%|          | 0/6 [00:00<?, ?it/s]

In [144]:
checkCondImpMonotoneIncreasing(G2perms)

[('1<0<2', '1,2,2,2'), ('1<0<2', '1,2,2,2,0')]

In [145]:
checkMonotoneIncreasingImpCond(G2perms)

[]

In [146]:
for i in G2perms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [164]:
checkDeltaPattern(G2perms)

In [148]:
checkHDeltasRepeat(G2perms)

[]

In [149]:
for i in generateEquivalence(G2perms):
    print(f"{i[0]} = {i[1]}")

## Type B

In [79]:
Bperms = genPermutations('B',6,2)

In [80]:
checkDeltaTypePrectionPerms(Bperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

[('B', '2<5<0<1<3<4<6', 6, 'cofac', 'last'),
 ('B', '2<5<0<1<3<6<4', 6, 'cofac', 'last'),
 ('B', '2<5<0<1<4<6<3', 6, 'cofac', 'last'),
 ('B', '2<5<0<1<4<3<6', 6, 'cofac', 'last'),
 ('B', '2<5<0<1<6<3<4', 6, 'cofac', 'last'),
 ('B', '2<5<0<1<6<4<3', 6, 'cofac', 'last'),
 ('B', '2<5<0<6<1<4<3', 6, 'cofac', 'last'),
 ('B', '2<5<0<6<1<3<4', 6, 'cofac', 'last'),
 ('B', '2<5<1<0<3<4<6', 6, 'cofac', 'last'),
 ('B', '2<5<1<0<3<6<4', 6, 'cofac', 'last'),
 ('B', '2<5<1<0<4<3<6', 6, 'cofac', 'last'),
 ('B', '2<5<1<0<4<6<3', 6, 'cofac', 'last'),
 ('B', '2<5<1<0<6<4<3', 6, 'cofac', 'last'),
 ('B', '2<5<1<0<6<3<4', 6, 'cofac', 'last'),
 ('B', '2<5<1<6<0<3<4', 6, 'cofac', 'last'),
 ('B', '2<5<1<6<0<4<3', 6, 'cofac', 'last'),
 ('B', '2<5<6<0<1<3<4', 6, 'cofac', 'last'),
 ('B', '2<5<6<0<1<4<3', 6, 'cofac', 'last'),
 ('B', '2<5<6<1<0<3<4', 6, 'cofac', 'last'),
 ('B', '2<5<6<1<0<4<3', 6, 'cofac', 'last'),
 ('B', '2<6<0<1<3<4<5', 5, 'cofac', 'last'),
 ('B', '2<6<0<1<3<4<5', 6, 'cofac', 'last'),
 ('B', '2<

In [64]:
writeDeltaTypesToFile(Bperms,"BDeltaTypes.csv",False)

  0%|          | 0/5040 [00:00<?, ?it/s]

In [114]:
checkCondImpMonotoneIncreasing(Bperms)

[('2<0<1<3<4<5', '2,3,4,5,5,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,1,0'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,3,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,1,0'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,3,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,3,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,3,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<5<3<4', '2,3,4,5,5,1'),
 ('2<0<1<5<3<4', '2,3,4,5,5,4,1'),
 ('2<0<1<5<3<4', '2,3,4,5,5,1,0'),
 ('2<0<1<5<3<4', '2,3,4,5,5,4,3,1

In [22]:
checkMonotoneIncreasingImpCond(Bperms)

[]

In [116]:
for i in Bperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [28]:
checkDeltaPattern(Bperms,printDelta=True)

4
2
SL_{5}(d)
34554SL_{3}(d)32102
34554SL_{3}(d)SL_{3}(d)32102
34554SL_{3}(d)SL_{3}(d)SL_{3}(d)32102
34554SL_{3}(d)SL_{3}(d)SL_{3}(d)SL_{3}(d)32102
3<0<1<4<2<5


In [118]:
checkHDeltasRepeat(Bperms)

[]

In [119]:
for i in generateEquivalence(Bperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<4<0<5 

## Type D

In [24]:
Dperms = genPermutations('D',7,2)

In [25]:
checkDeltaTypePrectionPerms(Dperms)

  0%|          | 0/40320 [00:00<?, ?it/s]

[]

In [66]:
writeDeltaTypesToFile(Dperms,"DDeltaTypes.csv")

  0%|          | 0/5040 [00:00<?, ?it/s]

In [130]:
checkMonotoneIncreasingImpCond(Dperms)

[('5<4<3<2<1<0', array([0, 0, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([0, 1, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([1, 1, 0, 0, 0, 1]))]

In [131]:
checkCondImpMonotoneIncreasing(Dperms)

[('2<0<4<5<1<3', '2,3,1,5,4,3'),
 ('2<0<4<5<1<3', '2,3,1,5,4,3,0'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3,0'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3,1'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3,1'),
 ('2<4<0<5<1<3', '2,3,1,5,4,3'),
 ('2<4<0<5<1<3', '2,3,1,5,0,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,1,4,3'),
 ('2<4<5<0<1<3', '2,3,1,5,4,3'),
 ('2<4<5<0<1<3', '2,3,1,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,1,5,4,3'),
 ('2<5<0<4<1<3', '2,3,1,4,5,3'),
 ('2<5<0<4<1<3', '2,3,1,4,0,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,1,5,3'),
 ('2<5<4<0<1<3', '2,3,1,4,5,3'),
 ('2<5<4<0<1<3', '2,3,1,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,1,4,5,3'),
 ('3<0<1<4<5<2', '3,2,5,1,0,2'),
 ('3<0<1<4<5<2', '3,2,5,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,5,1,0,2'),
 ('3<0<4<1<5<2', '3,2,5,1,0,2'),
 ('3<0<4<1<5<2'

In [42]:
checkDeltaPattern(Dperms,2,printDelta=True)

NameError: name 'Dperms' is not defined

In [133]:
for i in Dperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [134]:
checkHDeltasRepeat(Dperms)

[]

In [135]:
for i in generateEquivalence(Dperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<2<3<4<0<5 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<4<0<5
1<2<3<5<0<4 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<5<0<4
1<2<4<3<5<0 = 1<2<4<3<0<5
1<2<5<3<4<0 = 1<2<5<3<0<4
1<3<2<4<0<5 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<4<0<5
1<3<2<5<0<4 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<5<0<4
1<3<4<2<5<0 = 1<3<4<2<0<5
1<3<5<2<4<0 = 1<3<5<2<0<4
1<4<2<3<5<0 = 1<4<2<3<0<5
1<4<3<2<5<0 = 1<4<3<2<0<5
1<5<2<3<4<0 

## Type E

In [62]:
Eperms = genPermutations('E',7,3)

In [42]:
checkDeltaTypePrectionPerms(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

[('E', '0<3<1<2<5<6<4', 6, 'last', 'exceptional'),
 ('E', '0<3<1<2<6<4<5', 6, 'last', 'exceptional'),
 ('E', '0<3<1<2<6<5<4', 6, 'last', 'exceptional'),
 ('E', '0<3<1<4<5<6<2', 5, 'last', 'exceptional'),
 ('E', '0<3<1<4<6<5<2', 5, 'last', 'exceptional'),
 ('E', '0<3<1<4<6<2<5', 5, 'last', 'exceptional'),
 ('E', '0<3<1<5<2<6<4', 6, 'last', 'exceptional'),
 ('E', '0<3<1<5<4<6<2', 5, 'last', 'exceptional'),
 ('E', '0<3<1<5<6<2<4', 6, 'last', 'exceptional'),
 ('E', '0<3<1<5<6<4<2', 5, 'last', 'exceptional'),
 ('E', '0<3<1<6<2<4<5', 6, 'last', 'exceptional'),
 ('E', '0<3<1<6<2<5<4', 6, 'last', 'exceptional'),
 ('E', '0<3<1<6<4<5<2', 5, 'last', 'exceptional'),
 ('E', '0<3<1<6<4<2<5', 5, 'last', 'exceptional'),
 ('E', '0<3<1<6<5<2<4', 6, 'last', 'exceptional'),
 ('E', '0<3<1<6<5<4<2', 5, 'last', 'exceptional'),
 ('E', '0<3<2<1<5<6<4', 6, 'last', 'exceptional'),
 ('E', '0<3<2<1<6<4<5', 6, 'last', 'exceptional'),
 ('E', '0<3<2<1<6<5<4', 6, 'last', 'exceptional'),
 ('E', '0<3<2<5<1<6<4', 6, 'las

In [21]:
checkHDeltasRepeat(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

[]

In [40]:
for i in Eperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [63]:
writeDeltaTypesToFile(Eperms,"EDeltaTypes.csv")

  0%|          | 0/40320 [00:00<?, ?it/s]

In [46]:
for i in generateEquivalence(Eperms):
    print(f"{i[0]} = {i[1]}")

0<1<2<4<5<6<3 = 0<1<2<4<5<3<6
0<1<2<5<4<6<3 = 0<1<2<5<4<3<6
0<1<3<4<5<6<2 = 0<1<3<4<5<2<6
0<1<3<5<4<6<2 = 0<1<3<5<4<2<6
0<1<4<2<5<6<3 = 0<1<4<2<5<3<6
0<1<4<3<5<6<2 = 0<1<4<3<5<2<6
0<1<4<5<2<6<3 = 0<1<4<5<2<3<6
0<1<4<5<3<6<2 = 0<1<4<5<3<2<6
0<1<4<5<6<2<3 = 0<1<4<5<2<3<6
0<1<4<5<6<2<3 = 0<1<4<5<2<6<3
0<1<4<5<6<3<2 = 0<1<4<5<3<2<6
0<1<4<5<6<3<2 = 0<1<4<5<3<6<2
0<1<5<2<4<6<3 = 0<1<5<2<4<3<6
0<1<5<3<4<6<2 = 0<1<5<3<4<2<6
0<1<5<4<2<6<3 = 0<1<5<4<2<3<6
0<1<5<4<3<6<2 = 0<1<5<4<3<2<6
0<1<5<4<6<2<3 = 0<1<5<4<2<3<6
0<1<5<4<6<2<3 = 0<1<5<4<2<6<3
0<1<5<4<6<3<2 = 0<1<5<4<3<2<6
0<1<5<4<6<3<2 = 0<1<5<4<3<6<2
0<2<1<4<5<6<3 = 0<2<1<4<5<3<6
0<2<1<5<4<6<3 = 0<2<1<5<4<3<6
0<2<3<4<5<1<6 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<5<1<6
0<2<3<4<6<1<5 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<6<1<5
0<2<3<5<4<6<1 = 0<2<3<5<4<1<6
0<2<3<5<6<1<4 = 0<2<3<5<1<6<4
0<2<3<6<4<5<1 = 0<2<3<6<4<1<5
0<2<4<1<5<6<3 = 0<2<4<1<5<3<6
0<2<4<3<5<1<6 = 0<2<4<3<1<5<6
0<2<4<3<5<

KeyboardInterrupt: 

In [19]:
getMaxPeriodicity(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

(10, '2<0<1<4<6<5<3', array([1, 0, 1, 1, 1, 1, 0]))

## Type F

In [19]:
Fperms = genPermutations('F',4,2)

In [20]:
checkDeltaTypePrectionPerms(Fperms)

  0%|          | 0/120 [00:00<?, ?it/s]

[('F', '1<0<4<2<3', 2, 'last', 'exceptional'),
 ('F', '1<0<4<3<2', 2, 'last', 'exceptional'),
 ('F', '1<4<0<2<3', 3, 'last', 'exceptional'),
 ('F', '1<4<0<3<2', 3, 'last', 'exceptional'),
 ('F', '3<0<1<4<2', 3, 'last', 'exceptional'),
 ('F', '3<0<4<1<2', 3, 'last', 'exceptional'),
 ('F', '3<0<4<2<1', 3, 'last', 'exceptional'),
 ('F', '3<1<0<4<2', 3, 'last', 'exceptional'),
 ('F', '3<1<4<0<2', 3, 'last', 'exceptional'),
 ('F', '3<1<4<2<0', 3, 'last', 'exceptional'),
 ('F', '3<4<0<1<2', 4, 'last', 'exceptional'),
 ('F', '3<4<0<2<1', 4, 'last', 'exceptional'),
 ('F', '3<4<1<2<0', 4, 'last', 'exceptional'),
 ('F', '3<4<1<0<2', 4, 'last', 'exceptional'),
 ('F', '3<4<2<1<0', 4, 'last', 'exceptional'),
 ('F', '3<4<2<0<1', 4, 'last', 'exceptional')]

In [24]:
writeDeltaTypesToFile(Fperms,"FDeltaTypes.csv",onlyIfExceptional=False)

  0%|          | 0/120 [00:00<?, ?it/s]

In [22]:
checkMonotoneIncreasingImpCond(Fperms)

UnboundLocalError: cannot access local variable 'returnArr' where it is not associated with a value

In [ ]:
checkCondImpMonotoneIncreasing(Fperms)

[('1<0<2<3<4', '1,2,3,4,3,4,2'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,2'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,2,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,2'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,2,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,2'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,2,0'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4,3'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4,0'),
 ('1<0<3<

In [ ]:
checkDeltaPattern(Fperms,2,printDelta=True)

1<3<0<2<4
  SL_2(\delta): 1,2,3,4,3,4-1,2,0,3,3,2
  SL_2(2*\delta): 1,2,3,4,3,4,1,2,0,3,3,1,2,0-1,2,3,4,3,4,2,3,3,2
1<3<0<4<2
  SL_3(\delta): 1,2,3,4,3,4-1,2,0,3,3,2
  SL_3(2*\delta): 1,2,3,4,3,4,1,2,0,3,3,1,2,0-1,2,3,4,3,2,4,3,3,2
1<3<4<0<2
  SL_3(\delta): 1,2,3,4,3,4-1,2,0,3,3,2
  SL_3(2*\delta): 1,2,3,4,3,4,1,2,0,3,3,1,2,0-1,2,3,4,3,2,4,3,3,2
1<4<3<0<2
  SL_3(\delta): 1,2,3,3,4,4-1,2,0,3,3,2
  SL_3(2*\delta): 1,2,3,3,4,4,1,2,0,3,3,1,2,0-1,2,3,3,2,4,3,4,3,2
2<0<1<4<3
  SL_1(\delta): 2,3,4,1-2,3,4,1,0,2,3,3
  SL_1(2*\delta): 2,3,4,1,2,3,4,1,0-2,3,4,1,2,3,3,2,3,4,1,0,2,3,3
2<0<4<1<3
  SL_1(\delta): 2,3,1,4-2,3,1,4,0,2,3,3
  SL_1(2*\delta): 2,3,1,4,2,3,1,4,0-2,3,1,4,2,3,3,2,3,1,4,0,2,3,3
2<1<0<4<3
  SL_1(\delta): 2,3,4,1-2,3,4,1,0,2,3,3
  SL_1(2*\delta): 2,3,4,1,2,3,4,1,0-2,3,4,1,2,3,3,2,3,4,1,0,2,3,3
2<1<4<0<3
  SL_1(\delta): 2,3,4,1-2,3,4,1,0,2,3,3
  SL_1(2*\delta): 2,3,4,1,2,3,4,1,0-2,3,4,1,2,3,3,2,3,4,1,0,2,3,3
2<1<4<3<0
  SL_1(\delta): 2,3,4,1-2,3,4,1,0,2,3,3
  SL_1(2*\delta): 2,3,

In [ ]:
for i in Fperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
checkHDeltasRepeat(Fperms)

[]

In [ ]:
getMaxPeriodicity(Fperms)

(11, '1<0<3<4<2', array([0, 1, 2, 2, 0]))

In [ ]:
for i in generateEquivalence(Fperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1 = 0<2<3<1<4
0<3<2<4<1 = 0<3<2<1<4
1<2<3<0<4 = 1<2<0<3<4
1<2<3<4<0 = 1<2<0<3<4
1<2<3<4<0 = 1<2<3<0<4
1<2<4<0<3 = 1<2<0<4<3
1<2<4<3<0 = 1<2<0<4<3
1<2<4<3<0 = 1<2<4<0<3
1<3<2<4<0 = 1<3<2<0<4
1<3<4<0<2 = 1<3<0<4<2
1<4<2<3<0 = 1<4<2<0<3
2<0<3<4<1 = 2<0<3<1<4
2<1<3<0<4 = 2<1<0<3<4
2<1<3<4<0 = 2<1<0<3<4
2<1<3<4<0 = 2<1<3<0<4
2<1<4<0<3 = 2<1<0<4<3
2<1<4<3<0 = 2<1<0<4<3
2<1<4<3<0 = 2<1<4<0<3
2<3<0<4<1 = 2<3<0<1<4
2<3<1<4<0 = 2<3<1<0<4
2<3<4<0<1 = 2<3<0<1<4
2<3<4<0<1 = 2<3<0<4<1
2<3<4<1<0 = 2<3<1<0<4
2<3<4<1<0 = 2<3<1<4<0
2<4<1<3<0 = 2<4<1<0<3
3<0<2<4<1 = 3<0<2<1<4
3<1<2<4<0 = 3<1<2<0<4
3<1<4<0<2 = 3<1<0<4<2
3<2<0<4<1 = 3<2<0<1<4
3<2<1<4<0 = 3<2<1<0<4
3<2<4<0<1 = 3<2<0<1<4
3<2<4<0<1 = 3<2<0<4<1
3<2<4<1<0 = 3<2<1<0<4
3<2<4<1<0 = 3<2<1<4<0
4<1<2<3<0 = 4<1<2<0<3
4<2<1<3<0 = 4<2<1<0<3
